# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier

In [11]:
df = pd.read_csv('/datasets/users_behavior.csv')
print(df.sample(n=10, random_state=12345))
print(df.info())
print(df.describe())

      calls  minutes  messages   mb_used  is_ultra
1415   82.0   507.89      88.0  17543.37         1
916    50.0   375.91      35.0  12388.40         0
1670   83.0   540.49      41.0   9127.74         0
686    79.0   562.99      19.0  25508.19         1
2951   78.0   531.29      20.0   9217.25         0
654    53.0   478.18      78.0  20152.53         0
2827   73.0   582.47      33.0  12095.91         0
1466   31.0   172.10      25.0  31077.59         0
2223   28.0   222.21      30.0  22986.30         0
2639   68.0   523.56      14.0  18910.66         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  321

## Разбейте данные на выборки

In [3]:
#Разобьем выборки на признаки и цель
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

#Разобьем выборку на тренировочную, валидационную и тестовую в пропорции 60:20:20
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size = 0.4, random_state=123)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid,
                                                                            test_size=0.5, random_state=123)
#Проверка размера выборок
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

print(target_train[target_train == 0].count())

(1928, 4) (1928,)
(643, 4) (643,)
(643, 4) (643,)
1334


## Исследуйте модели

In [4]:
#Дерево решений

#Все параметры по умолчанию
model = tree.DecisionTreeClassifier(random_state=123)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
print('accuracy of model with all defaults =', accuracy_score(target_valid, predictions_valid))

#Влияние макс глубины на точность
best_max_depth = 0
best_accuracy = 0
best_tree_model = None
for depth in range(1,20):
    model = tree.DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = depth
        best_tree_model = model
    print('max_depth = {0} : {1}'.format(depth, accuracy))
print('Оптимальная макс. глубина: {0}, точность: {1}'.format(best_max_depth, best_accuracy))

#text_representation = tree.export_text(best_tree_model)
#print(text_representation)

accuracy of model with all defaults = 0.7465007776049767
max_depth = 1 : 0.7620528771384136
max_depth = 2 : 0.7900466562986003
max_depth = 3 : 0.80248833592535
max_depth = 4 : 0.8040435458786936
max_depth = 5 : 0.8227060653188181
max_depth = 6 : 0.8149300155520995
max_depth = 7 : 0.8118195956454122
max_depth = 8 : 0.80248833592535
max_depth = 9 : 0.8180404354587869
max_depth = 10 : 0.80248833592535
max_depth = 11 : 0.8149300155520995
max_depth = 12 : 0.8102643856920684
max_depth = 13 : 0.7884914463452566
max_depth = 14 : 0.7900466562986003
max_depth = 15 : 0.7713841368584758
max_depth = 16 : 0.7807153965785381
max_depth = 17 : 0.7698289269051322
max_depth = 18 : 0.7511664074650077
max_depth = 19 : 0.7573872472783826
Оптимальная макс. глубина: 5, точность: 0.8227060653188181


In [5]:
#Случайный лес

best_forest_model = None
best_accuracy = 0
best_max_depth = 0
best_n_est = 0
for depth in range (1,10):
    for est in range (1, 20):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_max_depth = depth
            best_n_est = est
            best_forest_model = model
            best_accuracy = accuracy
print('Оптимальная макс.глубина: {0}, оптимальное кол-во деревьев: {1}, точность: {2}'.format(best_max_depth, 
                                                                                              best_n_est, 
                                                                                              best_accuracy))

Оптимальная макс.глубина: 8, оптимальное кол-во деревьев: 17, точность: 0.833592534992224


In [6]:
#Логистическая регрессия

regressor_model = LogisticRegression(random_state=123, solver='liblinear')
regressor_model.fit(features_train, target_train)
accuracy = regressor_model.score(features_valid,target_valid) 
print('Точность модели логистической регрессии:', accuracy)

Точность модели логистической регрессии: 0.7200622083981337


**Вывод**

Наибольшую точность дала модель случайного леса: 0.83 при глубине = 8 и количестве деревьев = 17.
Наименьшую точность - логистическая регрессия (0.72)

## Проверьте модель на тестовой выборке

In [7]:
print('Точность обученной модели случайного леса на тестовой выборке:', best_forest_model.score(features_test, target_test))
print('Точность обученной модели логистической регрессии на тестовой выборке:', regressor_model.score(features_test, target_test))
print('Точность обученной модели дерева решений на тестовой выборке:', best_tree_model.score(features_test, target_test))

Точность обученной модели случайного леса на тестовой выборке: 0.7993779160186625
Точность обученной модели логистической регрессии на тестовой выборке: 0.6967340590979783
Точность обученной модели дерева решений на тестовой выборке: 0.7713841368584758


**Вывод**

Проверка на тестовой выборке показала, что точность моделей пропорциональна точности на валидационной, хотя и несколько ниже.

## (бонус) Проверьте модели на адекватность

Для проверки на адекватность возьмем тривиальные модели и сравним их показатели эффективности с показателями обученной нами модели.
В качестве примитивных моделей возьмем две: 
1. Модель, которая всегда предсказывает превалирующий в тренировочной выборке класс. Для нашей выборки это будет тариф "Смарт" (is_ultra = 0), тк она скошена в сторону Смарта: 1334 против 594 записей.
2. Модель, предсказывающая равномерное количество 0 и 1.

In [8]:
#Модель, всегда предсказывающая наиболее частый класс
frequent_classifier = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
print('Точность тривиальной модели, всегда предсказывающей Смарт:', frequent_classifier.score(features_test, target_test))

Точность тривиальной модели, всегда предсказывающей Смарт: 0.6936236391912908


In [9]:
#Модель, предсказывающая класс с вероятностью 50%
uniform_classifier = DummyClassifier(strategy='uniform').fit(features_train, target_train)
print('Точность тривиальной модели, предсказывающей случайный результат:', uniform_classifier.score(features_test, target_test))

Точность тривиальной модели, предсказывающей случайный результат: 0.5116640746500778


In [10]:
#Сравним confusion matrix всех трех моделей
predictions_test = best_tree_model.predict(features_test)
freq_predictions_test = frequent_classifier.predict(features_test)
uniform_predictions_test = uniform_classifier.predict(features_test)

print('Confusion matrix нашей модели:')
print(confusion_matrix(target_test, predictions_test))
print('Confusion matrix модели, всегда предсказывающей Смарт:')
print(confusion_matrix(target_test, freq_predictions_test))
print('Confusion matrix модели, предсказывающей случайный результат:')
print(confusion_matrix(target_test, uniform_predictions_test))

Confusion matrix нашей модели:
[[414  32]
 [115  82]]
Confusion matrix модели, всегда предсказывающей Смарт:
[[446   0]
 [197   0]]
Confusion matrix модели, предсказывающей случайный результат:
[[221 225]
 [ 94 103]]


**В нашей модели** количество True Negative случаев (реальный класс:0, предсказанный:0) = 414; False Positive (реальный:0, предсказанный:1) = 32; False Negative (реальный:1, предсказанный:0) = 115; True Positive(реальный:1, предсказанный:1): 82.

Отметим, что с угадыванием наиболее дорогостоящей Ультры дела обстоят у нашей модели не очень: кол-во False Negative кейсов, когда предсказали "Смарт", а надо было "Ультра", достаточно велико = 115 (а верной угаданной Ультры - 82 случая). А это может означать, что оператор не досчитается потенциальной прибыли. 
Показатель Recall (чувствительность) = 82/(82+115) = 0.41.
И по Recall мы проигрываем даже случайной модели: 96/197 = 0.48

## Вывод

Из трёх обученных моделей: Дерево решений, Случайный лис, Логистическая регрессия, наибольшую точность предсказаний (accuracy) на тестовой выборке удалось достичь с помощью модели **Случайный лес : 0.8**. Это на 11% выше точности тривиальной модели, всегда предсказывающей "Смарт" (0.69).
Однако при этом чувствительность нашей модели всего 0.41 против 0.48 у случайной.